In [3]:
from google.colab import drive
drive.mount('/content/drive');

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ROOT_PATH = "/content/drive/Shareddrives/Proyecto RecSys 2021-2/Proyecto"

# Model

In [26]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers.core import Dense, Dropout

In [27]:
SEQUENCE_LENGTH = 29
NUM_FEATURES = 1024

In [28]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)

        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )

        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

In [29]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)

        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads

        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim,
            dropout=0.3
        )

        self.dense_proj = keras.Sequential([
            Dense(dense_dim, activation=tf.nn.gelu),
            Dense(embed_dim)
        ])

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [30]:
def build_transformer_model(
    dense_dim,
    num_heads,
    classes,
    dropout=0
):
    pos_emb = PositionalEmbedding(
        sequence_length=SEQUENCE_LENGTH,
        output_dim=NUM_FEATURES,
    )

    transf_enc = TransformerEncoder(
        embed_dim=NUM_FEATURES,
        dense_dim=dense_dim,
        num_heads=num_heads
    )

    head = Dense(classes, activation='softmax')

    inputs = keras.Input(shape=(None, None))
    embedded = pos_emb(inputs)

    x = transf_enc(embedded)
    x = layers.GlobalMaxPooling1D()(x)
    x = Dropout(dropout)(x)

    outputs = head(x)

    model = keras.Model(inputs, outputs)
    return model

# Data loading

In [18]:
FEATURES_PATH = f"{ROOT_PATH}/features-and-labels/features_all.npy"
LABELS_PATH = f"{ROOT_PATH}/features-and-labels/labels_all.npy"

In [19]:
import numpy as np


with open(FEATURES_PATH, 'rb') as file:
    features = np.load(file)

with open(LABELS_PATH, 'rb') as file:
    labels = np.load(file)

In [20]:
features.shape

(32, 40, 29, 1024)

In [21]:
labels.shape

(32, 40)

In [22]:
TEST_SIZE = 10
NUM_VIDEOS = 40

np.random.seed(0)

idxs = list(range(NUM_VIDEOS))

TEST_VIDEO_IDXS = np.random.choice(idxs, TEST_SIZE, replace=False)

In [23]:
TEST_VIDEO_IDXS

array([22, 20, 25,  4, 10, 15, 28, 11, 18, 29])

In [24]:
NUM_USERS = 32

user_train_features_dict, user_train_labels_dict = {}, {}
user_test_features_dict, user_test_labels_dict = {}, {}

for user in range(NUM_USERS):
    user_features = features[user, :, :, :]
    user_labels = labels[user, :]

    user_test_labels = np.expand_dims(user_labels[TEST_VIDEO_IDXS], axis=1)
    user_test_features = user_features[TEST_VIDEO_IDXS, :, :]

    user_test_features_dict[user] = user_test_features
    user_test_labels_dict[user] = user_test_labels

In [52]:
print(user_test_labels.shape)

(10, 1)


---

# Matrix construction

In [31]:
individual_model_checkpoint_path = lambda user: f"{ROOT_PATH}/models/transformer/checkpoints/user_{user}_model.ckpt"

In [32]:
dense_dim = 4096
num_heads = 1
classes = 2
dropout = 0.4

base_model = build_transformer_model(
    dense_dim, 
    num_heads,
    classes,
    dropout=dropout
)

In [39]:
predicted_labels_dict = {}

for user in range(NUM_USERS):
    print(f"Generating predicted labels for model {user}...")
    
    model = base_model
    model.load_weights(individual_model_checkpoint_path(user))
    
    preds = model(user_test_features_dict[user])
    predicted_labels = tf.argmax(preds, axis=1)

    predicted_labels_dict[user] = predicted_labels

Generating predicted labels for model 0...
Generating predicted labels for model 1...
Generating predicted labels for model 2...
Generating predicted labels for model 3...
Generating predicted labels for model 4...
Generating predicted labels for model 5...
Generating predicted labels for model 6...
Generating predicted labels for model 7...
Generating predicted labels for model 8...
Generating predicted labels for model 9...
Generating predicted labels for model 10...
Generating predicted labels for model 11...
Generating predicted labels for model 12...
Generating predicted labels for model 13...
Generating predicted labels for model 14...
Generating predicted labels for model 15...
Generating predicted labels for model 16...
Generating predicted labels for model 17...
Generating predicted labels for model 18...
Generating predicted labels for model 19...
Generating predicted labels for model 20...
Generating predicted labels for model 21...
Generating predicted labels for model 22..

In [47]:
to_json_labels = {
    user: pred_labels.numpy().tolist() for user, pred_labels in predicted_labels_dict.items()
}

In [51]:
import json


with open(f"{ROOT_PATH}/matrix.json", 'w') as file:
    json.dump(to_json_labels, file, indent=4)